<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Example-system-commands" data-toc-modified-id="Example-system-commands-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Example system commands</a></span></li><li><span><a href="#Useful-links" data-toc-modified-id="Useful-links-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Useful links</a></span></li><li><span><a href="#Commands" data-toc-modified-id="Commands-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Commands</a></span><ul class="toc-item"><li><span><a href="#initialize-awscli" data-toc-modified-id="initialize-awscli-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>initialize awscli</a></span></li><li><span><a href="#list-job-queues" data-toc-modified-id="list-job-queues-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>list job queues</a></span></li><li><span><a href="#list-failed-jobs" data-toc-modified-id="list-failed-jobs-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>list failed jobs</a></span></li><li><span><a href="#use-case:-job-details-for-a-failed-job" data-toc-modified-id="use-case:-job-details-for-a-failed-job-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>use-case: job details for a failed job</a></span></li><li><span><a href="#use-case:-find-error-message-for-a-failed-job" data-toc-modified-id="use-case:-find-error-message-for-a-failed-job-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>use-case: find error message for a failed job</a></span></li></ul></li><li><span><a href="#Parking-lot" data-toc-modified-id="Parking-lot-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Parking lot</a></span><ul class="toc-item"><li><span><a href="#(bash)-command-array" data-toc-modified-id="(bash)-command-array-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>(bash) command array</a></span></li></ul></li></ul></div>

In [ ]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.GPAuthWidget(genepattern.register_session("https://gp-beta-ami.genepattern.org/gp", "", ""))

## Example system commands
I'm using this notebook to record system administration commands which can come in handy for managing our various GenePattern Server instances. Most examples are run locally as shell commands by way of the ```%%bash``` built-in magic command.

Hint: list all available magics
```
%lsmagic
```
Hint: display content in a colored block
```html
<div class="alert alert-block alert-info">
```

## Useful links
* See: https://docs.python.org/3/
* See: https://jupyter.readthedocs.io/en/latest/
* See: https://jupyter-notebook.readthedocs.io/en/latest/
* See: https://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Working%20With%20Markdown%20Cells.html
* See: https://datascience.ibm.com/docs/content/analyze-data/markd-jupyter.html
* See: https://blog.dominodatalab.com/lesser-known-ways-of-using-notebooks/, example of how to use 'bash'
* See: http://ipython.readthedocs.io/en/stable/interactive/magics.html
* See: https://docs.aws.amazon.com/cli/latest/userguide/controlling-output.html#controlling-output-filter
* See: http://jmespath.org/specification.html
* See: http://opensourceconnections.com/blog/2015/07/27/advanced-aws-cli-jmespath-query/

## Commands

### initialize awscli

In [ ]:
%%bash -e
# fail on first error
#   bash -e ...
#   set -e
source activate awscli
which aws

### list job queues

In [ ]:
%%bash -e
# list all job queues
aws batch describe-job-queues

### list failed jobs
Example command:
```bash
aws batch list-jobs --job-status FAILED --job-queue gpbeta-default
```

In [ ]:
# must set jobQueue as a python variable
jobQueue = "gpbeta-default"

In [ ]:
%%bash -e -s "$jobQueue"

# set bash variable from python variable
jobQueue="${1}"

# init awscli
source activate awscli

# list failed jobs
aws batch list-jobs \
  --job-queue $jobQueue \
  --job-status FAILED

### use-case: job details for a failed job
Given a GenePattern job number (e.g. 11813), make the necessary aws batch cli calls to track down the error message.

Example command:
```bash
aws batch list-jobs \
  --job-queue gpbeta-default \
  --job-status FAILED \
  --query 'jobSummaryList[?jobName==`GP_Job_11813`]'
```

Example command:
```bash
aws batch list-jobs \
  --job-queue gpbeta-default \
  --job-status FAILED \
  --query 'jobSummaryList[?ends_with(jobName,`_11813`) == `true`].{jobId:jobId}'
```

In [ ]:
# must set gpJobNo
gpJobNo = "11813"

In [144]:
%%bash -e -s "$gpJobNo"

# set bash variable from python variable
gpJobNo="${1}"
jobName="GP_Job_${gpJobNo}"

# init awscli
#source activate awscli

#
# step 1: get the aws batch jobId from the gp jobId
#
jobId=$(aws batch list-jobs \
  '--job-queue' 'gpbeta-default' \
  '--job-status' 'FAILED' \
  '--query' "jobSummaryList[?ends_with(jobName,\`_$gpJobNo\`) == \`true\`].{jobId:jobId}" \
  '--output' 'text')
echo jobId=$jobId
aws batch describe-jobs --jobs $jobId

# alternative queries
# take 1
#   '--query' 'jobSummaryList[?jobName==`GP_Job_11813`]'
# take 2
#   query="jobSummaryList[?jobName==\`$jobName\`]"
#   '--query' "${query}"

jobId=eee65873-3853-4d36-b97b-353e5a72a764
{
    "jobs": [
        {
            "jobName": "GP_Job_11813",
            "jobId": "eee65873-3853-4d36-b97b-353e5a72a764",
            "jobQueue": "arn:aws:batch:us-east-1:718039241689:job-queue/gpbeta-default",
            "status": "FAILED",
            "attempts": [
                {
                    "container": {
                        "containerInstanceArn": "arn:aws:ecs:us-east-1:718039241689:container-instance/377fb7ae-3468-464a-91eb-56e94c7f799d",
                        "taskArn": "arn:aws:ecs:us-east-1:718039241689:task/ede07b08-09cf-41c5-874f-4e4b514bc435",
                        "reason": "CannotPullContainerError: error pulling image configuration: Get https://docker-images-prod.s3.amazonaws.com/registry-v2/docker/registry/v2/blobs/sha256/0a/0a9b86cca7afdce9f18628f199d8c0819deaf801b8bf939d953318a5ca2a89e5/data?X-Amz-Algorithm=AWS4-HMAC-SHA",
                        "logStreamName": "R25_Generic/default/ede07b08-09cf-41c5-

## Parking lot
Parking lot for example code and other snippets.

### (bash) command array

In [ ]:
%%bash

# declare command line as an array
cmd=()

# add items to cmd
cmd=(${cmd[@]} "echo" "Hello, World!")

# print the command
echo "Command args ..."
printf '    %s\n' "${cmd[@]}"

# run the command
echo "Running command ..."
"${cmd[@]}"